<a href="https://colab.research.google.com/github/gabrielcastroo/apis_cloud_ibm_s2t_t2s/blob/main/MonitorParaTreinamentoIdiomas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#Instalando o IBM Watson
!pip install ibm_watson wget
#Instalando ffmpeg
!pip install ffmpeg-python

In [21]:
#Importando as bibliotecas
import json
from ibm_watson import TextToSpeechV1, SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [52]:
#Criando e estabelecendo as credenciais de autenticação para o uso da API Text to Speech
authenticatorTTS = IAMAuthenticator('ddoHqBgdpRs7JDmhGpWIDcLlr8_40yq4ARMDa4kU7vNY')
text_to_speech = TextToSpeechV1(
    authenticator=authenticatorTTS
)

text_to_speech.set_service_url('https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/b1a08a9f-f870-4570-ac69-70c91532c1f0')

#Lendo um determinado texto e sintetizando usando a API Text to Speech

text_to_synthesize_ptbr = 'Olá, Eu irei ser seu assistente para treinamento de idiomas, por favor ouça o audio na língua inglesa e tente repetir a frase:'

with open('synthesized_audio_T2S.wav', 'wb') as audio_file:
    audio_file.write(
        text_to_speech.synthesize(
            text_to_synthesize_ptbr,
            voice='pt-BR_IsabelaVoice',
            accept='audio/wav'        
        ).get_result().content)

In [93]:
#Criando um método para a frase que o usuário deve repetir
text_english = "hello world"

with open('synthesized_audio_T2S_english.wav', 'wb') as audio_file:
    audio_file.write(
        text_to_speech.synthesize(
            text_english,
            voice='en-US_HenryV3Voice',
            accept='audio/wav'        
        ).get_result().content)

In [33]:
#Criando as credenciais necessárias para a API Speech to Text
url_s2t = "https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/18a382da-04c7-4d60-8acc-b7f39823bf22"
iam_apikey_s2t = "SONw9EN0YoeXvKKW78MBfGKcorHMvGtNNMIVs2VKCAkJ"

In [34]:
authenticatorSTT = IAMAuthenticator(iam_apikey_s2t)
speech_to_text = SpeechToTextV1(authenticator=authenticatorSTT)
speech_to_text.set_service_url(url_s2t)
speech_to_text

In [25]:
#Usando audio do microfone para usar na API Speech to Text

#Importando as bibliotecas

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [73]:
#Salvando o retorno da função get_audio
audio, sr = get_audio()

In [74]:
#Importando as bibliotecas para salvar o audio como arquivo
from scipy.io.wavfile import write

In [75]:
#Chamando a função e salvando o audio como arquivo.wav
write("synthesized_audio.wav", sr, audio)

In [76]:
#Lendo o arquivo de audio gerado e passando ele para texto

filename='synthesized_audio.wav'

with open(filename, mode="rb")  as wav:
    response = speech_to_text.recognize(audio=wav, content_type='audio/wav', model='en-US_BroadbandModel')

In [90]:
#Mostrando o resultado do método Speech to Text
response.result

{'result_index': 0,
 'results': [{'alternatives': [{'confidence': 0.56,
     'transcript': 'hello world '}],
   'final': True}]}

In [78]:
#Importando o método json_normalize da biblioteca pandas
from pandas import json_normalize

#Transformando o resultado em json para melhorar a visualização
json_normalize(response.result['results'],"alternatives")
recognized_text = response.result['results'][0]["alternatives"][0]["transcript"]
type(recognized_text)

str

In [94]:
#Salvando palavras em uma variavel

founded_words = recognized_text.split(" ")

initial_words = text_english.split(" ")



print(founded_words)
print(initial_words)

['hello', 'world', '']
['hello', 'world']


In [98]:
equal_words = set(initial_words) & set(founded_words)
print(F'Palvras que pronunciou corrretamente: {equal_words}')

Palvras que pronunciou corrretamente: {'hello', 'world'}


In [102]:
wrong_words = set(initial_words) - set(founded_words)
print(wrong_words)

set()
